In [1]:
## REQUIRES AUTHENTICATION, OTHERWISE VERY SLOW

import os
from datetime import datetime
import re
from github import Github, RateLimitExceededException
from bs4 import BeautifulSoup
import requests
#from flask import Flask, request, jsonify
#from flask_cors import CORS
from requests.exceptions import RequestException
from retry import retry
#from github import Github

In [14]:
#g = Github(base_url="https://github.com/pybamm-team")
#repo = g.get_repo("PyBaMM")

In [2]:
class GithubRepoScraper:
    """Scrape GitHub repositories."""
    def __init__(self, repo_name, selected_file_types=None):
        if selected_file_types is None:
            selected_file_types = []
        self.repo_name = repo_name
        self.selected_file_types = selected_file_types

    @retry(RateLimitExceededException, tries=5, delay=2, backoff=2)
    def fetch_all_files(self):
        """Fetch all files from the GitHub repository."""
        def recursive_fetch_files(repo, contents):
            files_data = []
            for content_file in contents:
                if content_file.type == "dir":
                    files_data += recursive_fetch_files(repo, repo.get_contents(content_file.path))
                else:
                    # Check if file type is in selected file types
                    if any(content_file.name.endswith(file_type) for file_type in self.selected_file_types):
                        file_content = ""
                        file_content += f"\n'''--- {content_file.path} ---\n"

                        if content_file.encoding == "base64":
                            try:
                                file_content += content_file.decoded_content.decode("utf-8")
                            except UnicodeDecodeError: # catch decoding errors
                                file_content += "[Content not decodable]"
                        elif content_file.encoding == "none":
                            # Handle files with encoding "none" here
                            print(f"Warning: Skipping {content_file.path} due to unsupported encoding 'none'.")
                            continue
                        else:
                            # Handle other unexpected encodings here
                            print(f"Warning: Skipping {content_file.path} due to unexpected encoding '{content_file.encoding}'.")
                            continue

                        file_content += "\n'''"
                        files_data.append(file_content)
            return files_data
        github_instance = Github()
        repo = github_instance.get_repo(self.repo_name)
        contents = repo.get_contents("")
        files_data = recursive_fetch_files(repo, contents)
        return files_data

    def write_to_file(self, files_data):
        """Built .txt file with all of the repo's files"""
        timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
        filename = f"/app/data/{self.repo_name.replace('/', '_')}_{timestamp}.txt"
        with open(filename, "w", encoding='utf-8') as f:
            f.write(f"*GitHub Repository \"{self.repo_name}\"*\n")
            for file_data in files_data:
                f.write(file_data)
        return filename

    def clean_up_text(self, filename):
        """Remove line breaks after 2."""
        with open(filename, 'r', encoding='utf-8') as f:
            text = f.read()
        cleaned_text = re.sub('\n{3,}', '\n\n', text)
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(cleaned_text)

    def run(self):
        """Run RepoToText."""
        print("Fetching all files...")
        files_data = self.fetch_all_files()

        print("Writing to file...")
        filename = self.write_to_file(files_data)

        print("Cleaning up file...")
        self.clean_up_text(filename)

        print("Done.")
        return filename

In [3]:
def scrape(repo_name, selected_filetypes):
    """Scrape GitHub repositories."""
    scraper = GithubRepoScraper(repo_name, selected_file_types)
    filename = scraper.run()

    with open(filename, 'r', encoding='utf-8') as file:
        file_content = file.read()

    return jsonify({"response": file_content})

In [12]:
scraper = GithubRepoScraper("pybamm-team/PyBaMM", selected_file_types = ".md")
filename = scraper.run()

with open(filename, 'r', encoding='utf-8') as file:
    file_content = file.read()



Request GET /repos/pybamm-team/PyBaMM failed with 403: rate limit exceeded
Setting next backoff to 1574.849971s


Fetching all files...


Request GET /repos/pybamm-team/PyBaMM failed with 403: rate limit exceeded
Setting next backoff to 0s
Request GET /repos/pybamm-team/PyBaMM failed with 403: rate limit exceeded
Setting next backoff to 0s
Request GET /repos/pybamm-team/PyBaMM failed with 403: rate limit exceeded
Setting next backoff to 0s
Request GET /repos/pybamm-team/PyBaMM failed with 403: rate limit exceeded
Setting next backoff to 0s
Request GET /repos/pybamm-team/PyBaMM failed with 403: rate limit exceeded
Setting next backoff to 0s
Request GET /repos/pybamm-team/PyBaMM failed with 403: rate limit exceeded
Setting next backoff to 0s
Request GET /repos/pybamm-team/PyBaMM failed with 403: rate limit exceeded
Setting next backoff to 0s
Request GET /repos/pybamm-team/PyBaMM failed with 403: rate limit exceeded
Setting next backoff to 0s
Request GET /repos/pybamm-team/PyBaMM failed with 403: rate limit exceeded
Setting next backoff to 0s
Request GET /repos/pybamm-team/PyBaMM failed with 403: rate limit exceeded


RetryError: HTTPSConnectionPool(host='api.github.com', port=443): Max retries exceeded with url: /repos/pybamm-team/PyBaMM (Caused by ResponseError('too many 403 error responses'))